# DeepDream

We'll describe an implementation of DeepDream.

## Load Data

We'll load the example image of the Drosophila ssTEM dataset from https://figshare.com/articles/dataset/Segmented_anisotropic_ssTEM_dataset_of_neural_tissue/856713. Alternatively, you can download an image from the corresponding GiHub repository: http://github.com/unidesigner/groundtruth-drosophila-vnc.

Segmented anisotropic ssTEM dataset of neural tissue. Stephan Gerhard, Jan Funke, Julien Martel, Albert Cardona, Richard Fetter. figshare. Retrieved 16:09, Nov 20, 2013 (GMT) http://dx.doi.org/10.6084/m9.figshare.856713

We'll download a pretrained model (VGG16) and freeze all the weights.

In [ ]:
import torchvision.models as models
model = models.vgg16(pretrained=True)

model.eval()
model.requires_grad_(False)

print(model)

The VGG16 is pretrained on the ImageNet dataset and the inputs are normalized wih respect to the mean and standard deviation of the channels of this dataset.

In [ ]:
import numpy as np
mean_ds = np.array([0.485, 0.456, 0.406], dtype=np.float32)
std_ds = np.array([0.229, 0.224, 0.225], dtype=np.float32)

We'll define a function that perform loss maximization through gradient ascend. 

In [ ]:
def deepdream(im, test_layer, num_iterations=100, step_size=0.01):

    import torch
    from deepdream import fwd_hooks, preprocess, deprocess
    import numpy as np
    from PIL import Image

    image = preprocess(im, mean_ds, std_ds)
    
    low = torch.tensor((-mean_ds / std_ds).reshape(1, -1, 1, 1))
    high = torch.tensor(((1 - mean_ds) / std_ds).reshape(1, -1, 1, 1))

    for t in range(num_iterations):
 
        with fwd_hooks(test_layers) as fh:
            out = model(image)

        losses = [] 
        for f in fh.stored:
            losses.append( f.mean() )
        loss = torch.stack(losses).sum()
        loss.backward()

        image.data += step_size*((image.grad.data - torch.mean(image.grad.data) )/(torch.std(image.grad.data)+1e-8))
        image.grad.data.zero_()

        image.data.clamp_(low, high)

    return Image.fromarray(np.uint8(np.clip(deprocess(image.data.clone(), mean_ds, std_ds)*255,0,255)), 'RGB')

We'll apply deepdreams to the image, using as output the activations of the first ReLU.

In [ ]:
from deepdream import plot_dream
test_layers = [model.features[1]]
im_out  =  deepdream(im, test_layers, num_iterations=100, step_size=.01)
plot_dream(im, im_out)

By using deeper layer activations, we'll enhance features at larger scales. 

In [ ]:
from deepdream import plot_dream
test_layers = [model.features[18]] 
im_out  =  deepdream(im, test_layers, num_iterations=100, step_size=.01)
plot_dream(im, im_out)

We'll combine images at different resolutions using octaves.

In [ ]:
octave_scale = 1.4
im_oct = im
test_layers = [ model.features[18]]

for n in range(-2,3):
    im_oct = im_oct.resize((int(im.size[0]*(octave_scale**n)),int(im.size[1]*(octave_scale**n))))
    im_oct  =  deepdream(im_oct, test_layers, num_iterations=100, step_size=.01)
    im_oct = im_oct.resize(im.size)

plot_dream(im, im_oct)

We can furthermore use the output of several layers simultaneously to enhance multiple features.

In [ ]:
octave_scale = 1.4
im_oct = im

ind = [15, 18, 20, 22, 25, 27, 29]
test_layers = [ model.features[i] for i in ind ] 

for n in range(-2,3):
    im_oct = im_oct.resize((int(im.size[0]*(octave_scale**n)),int(im.size[1]*(octave_scale**n))))
    im_oct  =  deepdream(im_oct, test_layers, num_iterations=200, step_size=.01)
    im_oct = im_oct.resize(im.size)

plot_dream(im, im_oct)